In [2]:
import numpy as np 
from helpers import * 
from implementations import *
from other_functions import *

In [4]:
MAX_ROWS = 10000

x_data = np.genfromtxt('data/dataset/x_train.csv', delimiter=",", skip_header=1,  max_rows=MAX_ROWS)
y_data = np.genfromtxt('data/dataset/y_train.csv', delimiter=",", skip_header=1,  max_rows=MAX_ROWS)


y_data = y_data[:, 1]  # remove ids
y_data[y_data == -1] = 0  # set -1 to 0 
x_data = x_data[:, 1:]  # remove ids

#x_data[np.isnan(x_data)] = 0
#x_data = normalize(x_data)

In [7]:
np.sum(np.isnan(x_data)) / (np.multiply(x_data.shape[0],x_data.shape[1]))

0.4479517133956386

In [6]:
np.multiply(2,3)

6

In [3]:
# STEP 1
# Drop sparse features
def columns_to_remove(x_data,t):
    remove = np.array([])

    for j in range(x_data.shape[1]):   
        # focus: columns with too many nan values
        num_nan = np.sum(np.isnan(x_data[:,j]))
        per_nan = num_nan / x_data.shape[0]
        
        # focus: columns with too many 77/99 values
        num_2729 = np.sum(x_data[:,j] == 77) + np.sum(x_data[:,j] == 99)
        per_2729 = num_2729 / x_data.shape[0]
        
        # focus: columns with too many 77/99 values
        num_3739 = np.sum(x_data[:,j] == 777) + np.sum(x_data[:,j] == 999)
        per_3739 = num_3739 / x_data.shape[0]

        # focus: columns with too many 7777/9999 values
        num_4749 = np.sum(x_data[:,j] == 7777) + np.sum(x_data[:,j] == 9999)
        per_4749 = num_4749 / x_data.shape[0]

        # focus: columns with too many 777777/999999 values
        num_6769 = np.sum(x_data[:,j] == 777777) + np.sum(x_data[:,j] == 999999)
        per_6769 = num_6769 / x_data.shape[0]

        # find out which features to remove
        num_remove = num_nan + num_2729 + num_3739 + num_4749 + num_6769
        per_remove = num_remove / x_data.shape[0]
        if ( per_remove >= t ):
            remove = np.append(remove,j)
            
    return remove.astype(int)

def reduced_data(x_data):   # ex filtered_data
    t = 0.6     # threshold
    reduced_data = np.delete(x_data, columns_to_remove(x_data, t), 1)
    return reduced_data

# STEP 2
# Fix the misleading data
def find_index(str):
    with open('data/dataset/x_train.csv', newline='') as f:
        reader = csv.reader(f)
        row1 = next(reader)
    return row1.index(str)

def find_var(i):
    with open('data/dataset/x_train.csv', newline='') as f:
        reader = csv.reader(f)
        row1 = next(reader)
    return row1[i]

def fixed_data(x_data):
    for j in range(x_data.shape[1]):
        if (find_var(j) == 'PHYSHLTH') or (find_var(j) == 'MENTHLTH') or (find_var(j) == 'POORHLTH'):
            for i in range(x_data.shape[0]):
                if x_data[i][j] == 88:
                    x_data[i][j] = 0
        # TO DO 
        # other data
    return x_data

In [4]:
#print(reduced_data(x_data))
reduced_data = reduced_data(x_data)
#print(x_data[:,find_index('PHYSHLTH')])
print(find_index('PHYSHLTH'))

print(fixed_data(reduced_data(x_data))[:,find_index('PHYSHLTH')])

[ 5. 88. 77. ... 88.  5.  5.]
28
[1. 1. 1. ... 1. 1. 1.]


In [21]:
import numpy as np

def remove_outliers_from_matrix(x_data, max_deviations=2):
    # Calculate the mean for each column (axis=0)
    mean = np.mean(x_data, axis=0)
    
    # Calculate the standard deviation for each column
    std = np.std(x_data, axis=0)
    
    # Calculate the absolute deviation from the mean for each column
    distance_from_mean = np.abs(x_data - mean)
    
    # Create a mask for outliers for each column
    outlier_mask = distance_from_mean < max_deviations * std
    
    # Apply the mask to the original matrix to remove outliers
    x_data_cleaned = x_data[outlier_mask.all(axis=1)]
    
    return x_data_cleaned

# Example usage:
# Create a sample data matrix with outliers
data_matrix = np.array([[1, 1, 1],
                        [4, 5, 6],
                        [7, 8, 9],
                        [100, 200, 300],  # Outlier in each column
                        [10, 11, 12]])

# Remove outliers from the matrix using the function
cleaned_data = remove_outliers_from_matrix(data_matrix)

print("Original Data:")
print(data_matrix)
print("\nCleaned Data:")
print(cleaned_data)


Original Data:
[[  1   1   1]
 [  4   5   6]
 [  7   8   9]
 [100 200 300]
 [ 10  11  12]]

Cleaned Data:
[[  1   1   1]
 [  4   5   6]
 [  7   8   9]
 [100 200 300]
 [ 10  11  12]]


In [64]:
# Function
import numpy as np

def remove_outliers_from_matrix(x_data, max_deviations):
    # Calculate the mean and standard deviation for the entire matrix
    mean = np.mean(x_data, axis=0)
    print("mean = ", mean)
    std = np.std(x_data, axis=0)
    print("std = ",std)
    
    # Calculate the absolute deviation from the mean for the entire matrix
    distance_from_mean = []
    for i in range(x_data.shape[0]):
        distance_from_mean.append( np.abs(x_data[i] - mean))
    print("distance = ",distance_from_mean)

    distance_from_mean = np.array(distance_from_mean)

    print(max_deviations * std)


    # Create a mask to identify outliers based on the absolute deviation
    outlier_sample = distance_from_mean > max_deviations * std

    print(outlier_sample)
    
    # Apply the mask to the original matrix to remove outliers
    sample_to_drop = np.zeros(x_data.shape[0])
    for i in range(x_data.shape[0]):
        for j in range(x_data.shape[1]):
            if outlier_sample[i][j] == True:
                sample_to_drop[i] = True
    print(sample_to_drop)
    print(1 - sample_to_drop)
    
    x_data_cleaned = x_data[1 - sample_to_drop]
    
    return np.reshape(x_data_cleaned,(-1,x_data.shape[1]))

# Example usage:
# Create a sample data matrix with outliers
data_matrix = np.array([[1, 1, 1],
                        [4, 5, 6],
                        [7, 8, 9],
                        [100, 200, 300],  # Outlier in each column
                        [100, 110, 120]])

# Remove outliers from the matrix using the function
cleaned_data = remove_outliers_from_matrix(data_matrix,1)

print("Original Data:")
print(data_matrix)
print("\nCleaned Data:")
print(cleaned_data)


mean =  [42.4 64.8 87.2]
std =  [ 47.06846078  78.98708755 115.32458541]
distance =  [array([41.4, 63.8, 86.2]), array([38.4, 59.8, 81.2]), array([35.4, 56.8, 78.2]), array([ 57.6, 135.2, 212.8]), array([57.6, 45.2, 32.8])]
[ 47.06846078  78.98708755 115.32458541]
[[False False False]
 [False False False]
 [False False False]
 [ True  True  True]
 [ True False False]]
[0. 0. 0. 1. 1.]


TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''